# Cours TAL - Laboratoire 5<br/>Le modèle word2vec et ses applications 

**Objectif**

Le but de ce travail est de comparer un modèle word2vec pré-entraîné avec deux modèles que vous entraînerez vous-mêmes sur des jeux de tailles différentes.  La comparaison se fera sur une tâche de similarité de mots et sur une tâche de raisonnement par analogie, les deux en anglais.  Vous utiliserez la librairie Python Gensim qui offre des fonctions pour manipuler des vecteurs de mots. 

In [1]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath
from gensim import downloader as api
import scipy
import os
import psutil

## Tester en évaluer un modèle déjà entrainé sur Google News

In [2]:
# Télécharger le modèle pour la première fois
#w2v_vectors = api.load("word2vec-google-news-300") 

In [3]:
# Importer le modèle
path_to_file = "C:\\Users\\Vicky\\gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz"
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)

a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ? 

In [4]:
process = psutil.Process(os.getpid())
memory_usage = process.memory_info().rss / (1024 * 1024 * 1024)  # En GB

print(f"Utilisation mémoire : {memory_usage:.2f} GB")

Utilisation mémoire : 3.90 GB


b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?  

In [5]:
print(f"Dimension de l'espace vectoriel : {w2v_vectors.vector_size}")

Dimension de l'espace vectoriel : 300


c. Quelle est la taille du vocabulaire connu du modèle ?  Veuillez afficher cinq mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas. 

In [6]:
print(f"Taille du vocabulaire : {len(w2v_vectors.key_to_index)}")

Taille du vocabulaire : 3000000


In [7]:
words = [
    # Mot présents dans le vocabulaire
    'computer', 
    'apple', 
    'cat', 
    'gibberish', 
    'music', 

    # Mots absents du vocabulaire
    'ulotrichous', # Adj. 1. Having wooly hair (https://en.wiktionary.org/wiki/ulotrichous)
    'tittynope' # Noun 1. A small amount left over; a modicum. (https://en.wiktionary.org/wiki/tittynope)
]

for word in words:
    print(f'{word} : {'Mot présent' if word in w2v_vectors.key_to_index else 'Mot absent'}')

computer : Mot présent
apple : Mot présent
cat : Mot présent
gibberish : Mot présent
music : Mot présent
ulotrichous : Mot absent
tittynope : Mot absent


> Lors de nos tests, nous avons observé que plusieurs mots pensé comme potentiellement absents étaient en réalité connu par le modèle. Les mots inconnus sont donc à chercher dans des domaines spécialisés ou des mots désuets

d. Quelle est la similarité entre les mots rabbit et carrot ?  Veuillez rappeler comment on mesure les similarités entre deux mots grâce à leurs vecteurs.  

In [8]:
w1 = "rabbit"
w2 = "carrot"
print(f"La similarité entre '{w1}' et '{w2}' est de {w2v_vectors.similarity(w1, w2):.3f}")

La similarité entre 'rabbit' et 'carrot' est de 0.363


> La similarité est mesurée à l'aide de la distance cosinus entre les deux vecteurs, calculée avec la formule
> sim_cos
> $$
sim_{cosine}(a, b) = \frac{a \cdot b}{||a|| \cdot ||b||}
$$
> _Rappel : $a \cdot b$ est le produit scalaire entre les deux vecteurs. $||a||$ et $||b||$ sont leurs normes._
>
> Les valeurs de similarité varient donc entre -1 et 1. Un similarité de 1 signifie que les vecteurs sont identiques alors qu'une valeur de -1 indique qu'ils sont opposés. Une valeur de 0 indique que les vecteurs sont orthogonaux et donc que les mots sont sans rapport l'un avec l'autre.
> Dans notre cas, `rabbit` et `carrot` ont une similarité de 0.363, indiquant une certaine proximité dans l'espace vectoriel mais sans que ceux-ci soient réellement proche et interchangeables dans un phrase, comme pourraient l'être `rabbit` avec d'autres animaux (fox, possum, cat, squirrel, ...).

e. Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés.  Pour chaque paire, calculez la similarité entre les deux mots.  Veuillez indiquer si les similarités obtenues correspondent à vos intuitions sur la proximité des sens des mots.  

In [9]:
words = [
    # Words with similar senses => Should be a relatively high similarity
    ["sleepy", "somnolent"],
    ["kitten", "cat"],
    ["blue", "teal"],

    # Words with opposit senses, but that can be uses in same context => Should be around 0.3 - 0.5
    ["win", "lose"],
    ["light", "dark"],
    ["early", "late"],

    # Words with different senses, that cannot be used in same context => Should be a relatively low similarity
    ["book", "car"],
    ["William", "sea"],
    ["jewelry", "hunger"],
    
    # Other (Can be ambiguous words) => Will probably be a small similarity
    ["ray", "skate"] # Those are two cartilaginous fishes
]

for pair in words:
    print(f"La similarité entre '{pair[0]}' et '{pair[1]}' est de {w2v_vectors.similarity(pair[0], pair[1]):.3f}")

La similarité entre 'sleepy' et 'somnolent' est de 0.597
La similarité entre 'kitten' et 'cat' est de 0.746
La similarité entre 'blue' et 'teal' est de 0.636
La similarité entre 'win' et 'lose' est de 0.395
La similarité entre 'light' et 'dark' est de 0.471
La similarité entre 'early' et 'late' est de 0.812
La similarité entre 'book' et 'car' est de 0.128
La similarité entre 'William' et 'sea' est de -0.077
La similarité entre 'jewelry' et 'hunger' est de -0.000
La similarité entre 'ray' et 'skate' est de 0.104


> Parmis les paires considérées, les similarités correspondent assez bien avec nos intuitions, à l'exception du cas de `early` et `late`, qui sera discuté au point suivant.
> 
> Globalement, les mots de sens proche sont proches dans l'espace vectoriel et les mots de sens éloignés ont des similarités proche de 0. Malgré nos efforts pour trouver des paires de mots avec de grandes valeurs négatives, nous n'avons pas réussis à en trouver. Les mots les plus éloignés restent avec des valeurs négatives très faibles et sont difficile à trouver. Il est également intéressant de voir que des mots ayants plusieurs sens, comme c'est le cas pour `ray` et `skate` qui sont les noms de poissons catilagineux, sont utilisés dans le sens le plus courant et peuvent donc offrir des résultats surprenants.

f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ?  
Comment expliquez-vous cela ?  Est-ce une qualité ou un défaut du modèle word2vec ?

In [10]:
words = [
    ["blue", "red"],
    ["early", "late"]
]

for pair in words:
    print(f"La similarité entre '{pair[0]}' et '{pair[1]}' est de {w2v_vectors.similarity(pair[0], pair[1]):.3f}")

La similarité entre 'blue' et 'red' est de 0.723
La similarité entre 'early' et 'late' est de 0.812


> La question précédente nous avait permis de trouver la paire `early`-`late`, qui possède une similarité beaucoup plus élevée que ce à quoi l'on pourrait s'attendre. Bien que ces deux mots soient des antonymes, ils peuvent être utilisés dans des contextes très similaires. Après quelques recherches, nous avons également trouvé la paire `blue`-`red`, avec un score de similarité plus élevé qu'attendu.
>
> Word2Vec apprends des représentations vectorielles en se basant sur les contextes d'apparition des mots et non pas sur leur sens, il est donc normal que les paires comme `early`-`late` ou  `blue`-`red` possède des vecteurs similaires. Cet méthode d'apprentissage des vecteurs de mots permet au modèle de bien capter la manière dont une langue est utilisée (mots liés par leur utilisation) et permet de comprendre des similarités (analogie ou opposition) entre des mots. Si l'on cherche à avoir une distinction au niveau du sens (synonymes vs anthonymes), Word2Vec n'est pas assez précis et il faudra se tourner vers d'autres modèles, plus performants dans cette tâche.

g. En vous aidant de la [documentation de Gensim sur KeyedVectors](https://radimrehurek.com/gensim_3.8.3/models/keyedvectors.html), obtenez les scores du modèle word2vec sur les données de test WordSimilarity-353.  Veuillez rappeler en 1-2 phrases comment les différents scores sont calculés. 

In [11]:
eval_wordsim = w2v_vectors.evaluate_word_pairs(datapath("wordsim353.tsv"))

print(f"Corrélation de Pearson: {eval_wordsim[0][0]:.4f} (p-value: {eval_wordsim[0][1]:.4e})")
print(f"Corrélation de Spearman: {eval_wordsim[1][0]:.4f} (p-value: {eval_wordsim[1][1]:.4e})")
print(f"Ratio OOV: {eval_wordsim[2]}")

Corrélation de Pearson: 0.6239 (p-value: 1.7963e-39)
Corrélation de Spearman: 0.6589 (p-value: 2.5346e-45)
Ratio OOV: 0.0


> La méthode `evaluate_word_pairs()` compare la similarité cosinus entre les vecteurs de mots du modèle avec les scores de similarité humaine fournis dans le fichier wordsim353.tsv. Les scores retournées sont la corrélation de Pearson (Valeur entre -1 et 1 indiquant la corrélation linéaires entre les similarités prédites et attendues), la corrélation de Spearman (mesure la corrélation de rang, c'est à dire de l'ordre relatif des similarités) et le ratio OOV (indique la proportion de paires de mots contenant au moins un mot hors vocabulaire (Out-Of-Vocabulary)).

h. En vous aidant de la documentation, calculez le score du modèle word2vec sur les données questions-words.txt. Attention, cette évaluation prend une dizaine de minutes, donc il vaut mieux commencer par tester avec un fragment de ce fichier (copier/coller les 100 premières analogies).  Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [12]:
eval_qw = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
print(f"Score: {eval_qw[0]}")

Score: 0.7401448525607863


> La méthode `evaluate_word_analogies()` mesure la performance du modèle sur des tâches d'analogie de mots, où il doit trouver un mot D qui complète l'analogie "A est à B comme C est à D". Le score obtenu correspond à la proportion de bonnes réponses.

## 2. Entraîner deux nouveaux modèles word2vec à partir de deux corpus 

a. En utilisant gensim.downloader (voir question 1) récupérez le corpus qui contient les 108 premiers caractères de Wikipédia (en anglais) avec la commande : `corpus = api.load('text8')`. Combien de phrases et de mots (tokens) possède ce corpus ? 

In [13]:
corpus = api.load("text8")

print(f"Nombre de phrases dans le corpus : {api.info('text8')['num_records']}")
print(f"Nombre de mots (tokens) dans le corpus : {sum(len(sentence) for sentence in corpus)}")

Nombre de phrases dans le corpus : 1701
Nombre de mots (tokens) dans le corpus : 17005207


b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus (voir la [documentation de Word2vec](https://radimrehurek.com/gensim/models/word2vec.html)).  Si nécessaire, procédez progressivement, en commençant par utiliser 1% du corpus, puis 10%, etc., pour contrôler le temps nécessaire.    
• Veuillez indiquer la dimension choisie pour le embedding de ce nouveau modèle.  
• Combien de temps prend l’entraînement sur le corpus total ?   
• Quelle est la taille (en Mo) du modèle word2vec résultant ? 

In [14]:
%%time

t8_model = Word2Vec(sentences=corpus, vector_size=300)

CPU times: total: 3min 24s
Wall time: 1min 8s


> L'entrainement de ce modèle a donc pris 1min 8s, avec un temps total cumulé passé par tous les coeurs du processeur de 3 min 24s.

In [15]:
t8_model.save("word2vec_text8.model")

model_size = os.path.getsize("word2vec_text8.model") / (1024 * 1024)  # Mo

print(f"Dimension des embeddings : {t8_model.vector_size}")
print(f"Taille du modèle sauvegardé : {model_size:.2f} Mo")

Dimension des embeddings : 300
Taille du modèle sauvegardé : 2.08 Mo


c. Mesurez la qualité de ce modèle comme en (1g) et (1h).  Ce modèle est-il meilleur que celui entraîné sur Google News ?  Quelle est selon vous la raison de la différence ? 

In [16]:
eval_wordsim = t8_model.wv.evaluate_word_pairs(datapath("wordsim353.tsv"))

print(f"Corrélation de Pearson: {eval_wordsim[0][0]:.4f} (p-value: {eval_wordsim[0][1]:.4e})")
print(f"Corrélation de Spearman: {eval_wordsim[1][0]:.4f} (p-value: {eval_wordsim[1][1]:.4e})")
print(f"Ratio OOV: {eval_wordsim[2]}")

Corrélation de Pearson: 0.6095 (p-value: 4.4018e-37)
Corrélation de Spearman: 0.6271 (p-value: 9.2635e-40)
Ratio OOV: 0.56657223796034


In [17]:
eval_qw = t8_model.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print(f"Score: {eval_qw[0]}")

Score: 0.2598866887305772


> Le modèle entrainé sur le corpus Text8 est globalement moins performant que celui entrainé sur Google News. Bien que les scores de similarités soient légèrement plus faibles sur notre modèle, les p-values associées sont beaucoup plus grandes. Le modèle possède également un score beaucoup plus faible sur le test d'analogie, entre autre, dû au fait que beaucoup de termes ne sont pas connus.
>
> La taille du corpus Google News étant beaucoup plus grande (~40x) que celle de Text8, il est normal que le modèle Google News soit plus performant que le modèle Text8.

d. Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters [fourni par l’enseignant et appelé wikipedia_augmented.zip](https://drive.switch.ch/index.php/s/iYI9yB0PpL7iUFS) (à décompresser en un fichier ‘.dat’ de 413 Mo). Entraînez un nouveau modèle word2vec sur ce corpus, en précisant la dimension choisie pour les embeddings.   

• Utilisez la classe `Text8Corpus()` pour charger ce corpus, ce qui fera automatiquement la tokenisation et la segmentation en phrases.    
• Combien de temps prend l’entraînement ?    
• Quelle est la taille (en Mo) du modèle word2vec résultant ? 

In [18]:
corpus_augmented = Text8Corpus('data/wikipedia_augmented.dat')

In [19]:
%%time

t8_extended_model = Word2Vec(sentences=corpus_augmented, vector_size=300)

CPU times: total: 14min 58s
Wall time: 5min 1s


> L'entrainement de ce modèle a donc pris 5min 1s, avec un temps total cumulé passé par tous les coeurs du processeur de 14min 58s.

In [20]:
t8_extended_model.save("word2vec_text8_extended.model")

model_size = os.path.getsize("word2vec_text8_extended.model") / (1024 * 1024)  # Mo

print(f"Dimension des embeddings : {t8_model.vector_size}")
print(f"Taille du modèle sauvegardé : {model_size:.2f} Mo")

Dimension des embeddings : 300
Taille du modèle sauvegardé : 3.71 Mo


e. Testez ce modèle comme en (1g) et (1h).  Est-il meilleur que le précédent ?

In [21]:
eval_wordsim = t8_extended_model.wv.evaluate_word_pairs(datapath("wordsim353.tsv"))

print(f"Corrélation de Pearson: {eval_wordsim[0][0]:.4f} (p-value: {eval_wordsim[0][1]:.4e})")
print(f"Corrélation de Spearman: {eval_wordsim[1][0]:.4f} (p-value: {eval_wordsim[1][1]:.4e})")
print(f"Ratio OOV: {eval_wordsim[2]}")

Corrélation de Pearson: 0.5112 (p-value: 6.6893e-25)
Corrélation de Spearman: 0.5230 (p-value: 3.5533e-26)
Ratio OOV: 0.0


In [22]:
eval_qw = t8_extended_model.wv.evaluate_word_analogies(datapath('questions-words.txt'))
print(f"Score: {eval_qw[0]}")

Score: 0.3531107243854688


> Le modèle avec le corpus Text8 augmenté est meilleur sur le test d'analogie que le modèle Text8, mais reste beaucoup moins bon que le modèle Google News. L'impact de l'augmentation du nombre de tokens est facilement visible grâce au ratio OOV: tout les mots du test d'analogie sont connus, contrairement pour le modèle Text8 de base. Les scores de similarités sont moins bons que ceux obtenus par le modèle Text8, avec des p-values bien plus grandes également, ce qui indique une corrélation moins forte entre les vecteurs de mots.
>
> Le corpus utilisé pour ce modèle est une version étendue du coprus Text8, enrichi avec les dépêches écononiques de Reuters. Le corpus possède donc beaucoup plus de tokens et réussis à couvrir l'ensemble des mots utilisés dans le test d'analogie. Les vecteurs de mots résultants sont donc influencés par cet ajout et sont donc différents de ceux obtenus avec le corpus Text8 de base, entrainant une différence dans les scores de similarité. Le volume et le contexte des données d'entrainement influencent donc grandement les performances du modèle avec les tests de Word2Vec